In [36]:
import geopandas as gpd
import pandas as pd
import ee
from src.data import mtbs
from src.data import raster
import rasterio as rio
import matplotlib.pyplot as plt
from shapely.geometry import Point
from rasterio.plot import show
pd.options.mode.chained_assignment = None  # default='warn'
import seaborn as sns
sns.set_theme()

import importlib
importlib.reload(raster)


<module 'src.data.raster' from '/home/jk871/fire-regen/src/data/raster.py'>

In [ ]:
ee.Authenticate()

In [2]:
ee.Initialize()

In [37]:
burn_raster = raster.RasterSampler(raster.BURN_DATA_RASTER, raster.BURN_RASTER_BANDS)
lc_raster = raster.RasterSampler(raster.LAND_COVER_RASTER, raster.LAND_COVER_BANDS)

In [5]:
gedi_shots = pd.read_csv('/maps/fire-regen/data/sierras_gedi_shots.csv', index_col=0)

In [35]:
gedi_matched = burn_raster.sample_2x2(gedi_shots, 'lon_lowestmode', 'lat_lowestmode')

In [39]:
gedi_matched.to_csv(f'/maps/fire-regen/data/sierras_matched_lc_all_2x2.csv')

In [38]:
gedi_matched = lc_raster.sample_2x2(gedi_matched, 'lon_lowestmode', 'lat_lowestmode')

In [33]:
gedi_matched

,shot_number,absolute_time,lon_lowestmode,lat_lowestmode,agbd,agbd_pi_lower,agbd_pi_upper,agbd_se,l2_quality_flag,l4_quality_flag,...,burn_severity_std,burn_severity_median,burn_year_3x3,burn_year_mean,burn_year_std,burn_year_median,burn_counts_3x3,burn_counts_mean,burn_counts_std,burn_counts_median
0,184890800200086632,2022-03-19 02:41:53.252278+00:00,-117.983310,37.034945,19.594500,2.419877,53.254089,2.981980,1,1,...,0.000000,0.0,"[0, 0, 0, 0]",0.0,0.0,0.0,"[0, 0, 0, 0]",0.00,0.000000,0.0
1,184891100200081451,2022-03-19 02:41:53.339050+00:00,-117.985725,37.040705,17.788033,1.812951,50.249866,2.982302,1,1,...,0.000000,0.0,"[0, 0, 0, 0]",0.0,0.0,0.0,"[0, 0, 0, 0]",0.00,0.000000,0.0
2,184891100200081452,2022-03-19 02:41:53.347314+00:00,-117.985239,37.041043,16.219337,1.336309,47.591499,2.982735,1,1,...,0.000000,0.0,"[0, 0, 0, 0]",0.0,0.0,0.0,"[0, 0, 0, 0]",0.00,0.000000,0.0
3,184891100200081453,2022-03-19 02:41:53.355578+00:00,-117.984755,37.041380,6.750648,-9999.000000,29.955231,2.990220,1,1,...,0.000000,0.0,"[0, 0, 0, 0]",0.0,0.0,0.0,"[0, 0, 0, 0]",0.00,0.000000,0.0
4,184891100200081454,2022-03-19 02:41:53.363842+00:00,-117.984267,37.041720,4.370386,-9999.000000,24.677294,2.994677,1,1,...,0.000000,0.0,"[0, 0, 0, 0]",0.0,0.0,0.0,"[0, 0, 0, 0]",0.00,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27629560,145740600300499241,2021-07-09 14:28:27.680490+01:00,-120.454065,40.766560,4.699281,-9999.000000,25.447033,2.993938,1,1,...,0.000000,2.0,"[1990, 1990, 1990, 1990]",1990.0,0.0,1990.0,"[1, 1, 1, 1]",1.00,0.000000,1.0
27629561,145740600300499242,2021-07-09 14:28:27.688756+01:00,-120.453532,40.766257,2.924632,-9999.000000,21.060156,2.998635,1,1,...,0.433013,2.0,"[1990, 1990, 1990, 1990]",1990.0,0.0,1990.0,"[0, 1, 1, 1]",0.75,0.433013,1.0
27629562,145740600300499244,2021-07-09 14:28:27.705288+01:00,-120.452463,40.765649,1.500722,-9999.000000,16.865877,3.004451,1,1,...,0.433013,2.0,"[1990, 1990, 1990, 1990]",1990.0,0.0,1990.0,"[1, 1, 1, 0]",0.75,0.433013,1.0
27629564,145740800300301239,2021-07-09 14:28:27.978045+01:00,-120.463172,40.762280,3.823108,-9999.000000,23.357908,2.996024,1,1,...,0.000000,0.0,"[0, 0, 0, 0]",0.0,0.0,0.0,"[0, 0, 0, 0]",0.00,0.000000,0.0


In [9]:
gedi_matched_old = pd.read_csv(f'/maps/fire-regen/data/sierras_matched_lc_all.csv', index_col=0)

In [29]:
gedi_matched_old[gedi_matched_old.burn_severity_std!=0].shape[0]

1421293

In [30]:
gedi_matched[gedi_matched.burn_severity_std!=0].shape[0]

949576

In [32]:
gedi_matched_old.shape[0]

9884257

In [13]:
gedi_old = gedi_matched_old[gedi_matched_old.shot_number.isin(gedi_matched.shot_number)]

In [22]:
gedi_old[gedi_old.shot_number == 141320500300286362][['burn_severity_3x3', 'burn_year_3x3', 'burn_counts_3x3', 'burn_sev']]

,burn_severity_3x3,burn_year_3x3,burn_counts_3x3
8439,[2 2 1 2 2 1 2 0 0],[2017 2017 2017 2017 2017 2017 2017 0 0],[1 1 0 1 1 0 1 0 0]


In [24]:
gedi_old[gedi_old.burn_severity_std != 0][['shot_number', 'burn_severity_3x3', 'burn_year_3x3', 'burn_counts_3x3']]

,shot_number,burn_severity_3x3,burn_year_3x3,burn_counts_3x3
8439,141320500300286362,[2 2 1 2 2 1 2 0 0],[2017 2017 2017 2017 2017 2017 2017 0 0],[1 1 0 1 1 0 1 0 0]
8440,141320500300286363,[1 0 0 0 0 0 0 0 0],[2017 0 0 0 0 0 0 0 0],[0 0 0 0 0 0 0 0 0]
8444,141320600300524419,[1 0 0 2 0 0 2 1 0],[2017 0 0 2017 0 0 2017 2017 0],[0 0 0 1 0 0 1 0 0]
8455,28901100300221100,[2 0 0 2 1 0 2 2 0],[2017 0 0 2017 2017 0 2017 2017 0],[1 0 0 1 0 0 1 1 0]
8484,94310800200083134,[2 2 2 1 0 2 0 0 0],[2017 2017 2017 2017 0 2017 0 0 0],[1 1 1 0 0 1 0 0 0]
8514,141320500300286314,[2 0 0 2 0 0 0 0 0],[1996 0 0 1996 0 0 0 0 0],[1 0 0 1 0 0 0 0 0]
8543,169790600200452949,[2 0 0 2 0 0 2 0 0],[1996 0 0 1996 0 0 1996 0 0],[1 0 0 1 0 0 1 0 0]
8544,169790600200452951,[2 2 2 2 2 0 2 2 0],[1996 1996 1996 1996 1996 0 1996 1996 0],[1 1 1 1 1 0 1 1 0]
8547,169791100200077420,[2 1 0 1 1 0 1 1 1],[2017 2017 0 2017 2017 0 2017 2017 2017],[1 0 0 0 0 0 0 0 0]


In [25]:
gedi_matched[gedi_matched.burn_severity_std != 0][['shot_number', 'burn_severity_3x3', 'burn_year_3x3', 'burn_counts_3x3']]

,shot_number,burn_severity_3x3,burn_year_3x3,burn_counts_3x3
30141,141320500300286362,"[2, 1, 2, 1]","[2017, 2017, 2017, 2017]","[1, 0, 1, 0]"
30142,141320500300286363,"[0, 0, 1, 2]","[0, 0, 2017, 2017]","[0, 0, 0, 1]"
30148,141320600300524420,"[0, 1, 0, 1]","[0, 2017, 0, 2017]","[0, 0, 0, 0]"
30300,94310800200083134,"[1, 2, 2, 2]","[2017, 2017, 2017, 2017]","[0, 1, 1, 1]"
30350,48550800200082813,"[0, 0, 0, 2]","[0, 0, 0, 2017]","[0, 0, 0, 1]"
30473,169790600200452951,"[2, 2, 2, 0]","[1996, 1996, 1996, 0]","[1, 1, 1, 0]"
30482,169791100200077420,"[1, 0, 1, 0]","[2017, 0, 2017, 0]","[0, 0, 0, 0]"


In [2]:
i = 0
for chunk in pd.read_csv('/maps/fire-regen/data/sierras_gedi_shots.csv', index_col=0, chunksize=500000):
    chunk = chunk.drop(columns=['geometry', 'l2_quality_flag', 'l4_quality_flag', 'degrade_flag'])
    print(f'Processing chunk {i}.')
    chunk_with_burn = burn_raster.sample_2x2(chunk, 3)
    chunk_gpd = gpd.GeoDataFrame(chunk_with_burn, geometry=gpd.points_from_xy(chunk_with_burn.lon_lowestmode, chunk_with_burn.lat_lowestmode), crs=4326)
    chunk_gpd['gedi_year'] = pd.to_datetime(chunk_gpd.absolute_time, utc=True).dt.year
    chunk_gpd['time_since_burn'] = chunk_gpd.gedi_year - chunk_gpd.burn_year_sample 
    chunk_gpd.to_csv(f'/maps/fire-regen/data/sierras_matched_{i}.csv')
    i+=1
#gedi_with_burn.head(5)

KeyboardInterrupt: 

In [9]:
sampler = sample_raster.RasterSampler(sample_raster.LAND_COVER_RASTER_FILE)

In [14]:
i = 0
for chunk in pd.read_csv('/maps/fire-regen/data/sierras_matched_all.csv', index_col=0, chunksize=500000):
    print(f'Processing chunk {i}.')
    chunk_with_land_cover = sampler.sample(chunk, 3)
    chunk_with_land_cover.to_csv(f'/maps/fire-regen/data/sierras_matched_lc_{i}.csv')
    i+=1
#gedi_with_burn.head(5)

Processing chunk 0.
Processing chunk 1.
Processing chunk 2.
Processing chunk 3.
Processing chunk 4.
Processing chunk 5.
Processing chunk 6.
Processing chunk 7.
Processing chunk 8.
Processing chunk 9.
Processing chunk 10.
Processing chunk 11.
Processing chunk 12.
Processing chunk 13.
Processing chunk 14.
Processing chunk 15.
Processing chunk 16.
Processing chunk 17.
Processing chunk 18.
Processing chunk 19.


In [13]:
chunk_with_land_cover

,shot_number,absolute_time,lon_lowestmode,lat_lowestmode,agbd,agbd_pi_lower,agbd_pi_upper,agbd_se,beam_type,sensitivity,...,burn_counts_std,burn_counts_median,geometry,gedi_year,time_since_burn,land_cover_3x3,land_cover_sample,land_cover_mean,land_cover_std,land_cover_median
0,184890800200086632,2022-03-19 02:41:53.252278+00:00,-117.983310,37.034945,19.594500,2.419877,53.254089,2.981980,full,0.959151,...,0.0,0.0,POINT (-117.9833095044786 37.0349449804212),2022,2022,"[5, 5, 5, 5, 5, 5, 5, 5, 5]",5,5.000000,0.000000,5.0
1,184891100200081451,2022-03-19 02:41:53.339050+00:00,-117.985725,37.040705,17.788033,1.812951,50.249866,2.982302,full,0.966117,...,0.0,0.0,POINT (-117.98572485904388 37.04070455654646),2022,2022,"[5, 5, 5, 5, 5, 5, 5, 5, 5]",5,5.000000,0.000000,5.0
2,184891100200081452,2022-03-19 02:41:53.347314+00:00,-117.985239,37.041043,16.219337,1.336309,47.591499,2.982735,full,0.966954,...,0.0,0.0,POINT (-117.98523879930067 37.04104319818968),2022,2022,"[5, 5, 5, 5, 5, 5, 5, 5, 5]",5,5.000000,0.000000,5.0
3,184891100200081453,2022-03-19 02:41:53.355578+00:00,-117.984755,37.041380,6.750648,-9999.000000,29.955231,2.990220,full,0.970945,...,0.0,0.0,POINT (-117.98475524347532 37.04138016830484),2022,2022,"[5, 5, 5, 5, 5, 5, 5, 5, 5]",5,5.000000,0.000000,5.0
4,184891100200081454,2022-03-19 02:41:53.363842+00:00,-117.984267,37.041720,4.370386,-9999.000000,24.677294,2.994677,full,0.961078,...,0.0,0.0,POINT (-117.98426729218338 37.04172008473891),2022,2022,"[5, 5, 5, 5, 5, 5, 5, 5, 5]",5,5.000000,0.000000,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,132280800200194067,2021-04-13 17:57:22.890742+01:00,-118.052501,37.204197,7.294233,-9999.000000,31.084440,2.989434,full,0.954495,...,0.0,0.0,POINT (-118.0525005175952 37.204197494249264),2021,2021,"[5, 5, 5, 5, 5, 5, 5, 5, 5]",5,5.000000,0.000000,5.0
4996,132280800200194069,2021-04-13 17:57:22.907274+01:00,-118.051527,37.204868,27.002604,5.407206,65.084007,2.982169,full,0.956186,...,0.0,0.0,POINT (-118.05152749040516 37.20486810068212),2021,2021,"[7, 7, 7, 7, 7, 7, 7, 7, 7]",7,7.000000,0.000000,7.0
4997,132280800200194073,2021-04-13 17:57:22.940338+01:00,-118.049583,37.206210,1.331117,-9999.000000,16.288925,3.005378,full,0.957654,...,0.0,0.0,POINT (-118.04958280490946 37.206210246558136),2021,2021,"[5, 7, 7, 5, 7, 5, 5, 5, 5]",7,5.666667,0.942809,5.0
4998,132280800200194080,2021-04-13 17:57:22.998187+01:00,-118.046183,37.208561,0.970350,-9999.000000,14.964243,3.007645,full,0.961489,...,0.0,0.0,POINT (-118.04618298023622 37.20856104476237),2021,2021,"[5, 5, 5, 5, 5, 5, 5, 5, 5]",5,5.000000,0.000000,5.0


In [26]:
i = 0
for chunk in pd.read_csv('/maps/fire-regen/data/sierras_gedi_shots.csv', index_col=0, chunksize=500000):
    chunk = chunk.drop(columns=['geometry', 'l2_quality_flag', 'l4_quality_flag', 'degrade_flag'])
    print(f'Processing chunk {i}.')
    chunk_with_burn = sample_raster.sample_raster(chunk, 3)
    chunk_gpd = gpd.GeoDataFrame(chunk_with_burn, geometry=gpd.points_from_xy(chunk_with_burn.lon_lowestmode, chunk_with_burn.lat_lowestmode), crs=4326)
    chunk_gpd['gedi_year'] = pd.to_datetime(chunk_gpd.absolute_time, utc=True).dt.year
    chunk_gpd['time_since_burn'] = chunk_gpd.gedi_year - chunk_gpd.burn_year_sample 
    chunk_gpd.to_csv(f'/maps/fire-regen/data/sierras_matched_{i}.csv')
    i+=1
#gedi_with_burn.head(5)

Processing chunk 0.
Processing chunk 1.
Processing chunk 2.
Processing chunk 3.
Processing chunk 4.
Processing chunk 5.
Processing chunk 6.
Processing chunk 7.
Processing chunk 8.
Processing chunk 9.
Processing chunk 10.
Processing chunk 11.
Processing chunk 12.
Processing chunk 13.
Processing chunk 14.
Processing chunk 15.
Processing chunk 16.
Processing chunk 17.
Processing chunk 18.
Processing chunk 19.


In [3]:
# load chunks
gpds = []
for i in range(20):
    print(f'Processing chunk {i}')
    chunk = pd.read_csv(f'/maps/fire-regen/data/sierras_matched_lc_{i}.csv', index_col=0)
    gpds.append(chunk)
print('Joining dataframes')
gedi_sierras = gpd.GeoDataFrame( pd.concat(gpds, ignore_index=True))
print('Writing csv file')
gedi_sierras.to_csv(f'/maps/fire-regen/data/sierras_matched_lc_all.csv')

Processing chunk 0
Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Processing chunk 6
Processing chunk 7
Processing chunk 8
Processing chunk 9
Processing chunk 10
Processing chunk 11
Processing chunk 12
Processing chunk 13
Processing chunk 14
Processing chunk 15
Processing chunk 16
Processing chunk 17
Processing chunk 18
Processing chunk 19
Joining dataframes
Writing csv file


In [5]:
gedi_sierras.head(2)

,shot_number,absolute_time,lon_lowestmode,lat_lowestmode,agbd,agbd_pi_lower,agbd_pi_upper,agbd_se,beam_type,sensitivity,...,burn_year_std,burn_severity_median,burn_counts_3x3,burn_counts_sample,burn_counts_mean,burn_counts_std,burn_counts_median,geometry,gedi_year,time_since_burn
0,184890800200086632,2022-03-19 02:41:53.252278+00:00,-117.983310,37.034945,19.594500,2.419877,53.254089,2.981980,full,0.959151,...,0.0,0.0,[0 0 0 0 0 0 0 0 0],0,0.0,0.0,0.0,POINT (-117.9833095044786 37.0349449804212),2022,2022
1,184891100200081451,2022-03-19 02:41:53.339050+00:00,-117.985725,37.040705,17.788033,1.812951,50.249866,2.982302,full,0.966117,...,0.0,0.0,[0 0 0 0 0 0 0 0 0],0,0.0,0.0,0.0,POINT (-117.98572485904388 37.04070455654646),2022,2022


In [4]:
gedi_burn_gpd = gedi_sierras

### Match GEDI shots with terrain